In [2]:
from mintalib import core
from mintalib import functions as fx
from mintalib.samples import sample_prices


class PricesMethods:
    _prices = None

    def dispatch_prices(self, func, *args, **kwds):
        result = func(self._prices, *args, **kwds)
        return core.wrap_result(result, self._prices)

    def atr(self, period: int = 14):
        kwds = dict(period=period)
        return self.dispatch_prices(core.calc_atr, **kwds)



class SeriesMethods:
    _series = None

    def dispatch_series(self, func, *args, **kwds):
        result = func(self._series, *args, **kwds)
        return core.wrap_result(result, self._series)

    def ema(self, period: int = 20):
        kwds = dict(period=period)
        return self.dispatch_series(core.calc_ema, **kwds)

    def sma(self, period: int = 20):
        kwds = dict(period=period)
        return self.dispatch_series(core.calc_sma, **kwds)


def register_pandas(name="ta"):
    import pandas as pd

    @pd.api.extensions.register_dataframe_accessor(name)
    class PricesAccessor(PricesMethods):
        def __init__(self, prices):
            self._validate(prices)
            self._prices = prices

        @staticmethod
        def _validate(prices):
            if "close" not in prices.columns:
                raise AttributeError("Prices must have a 'close' column") 

        @property
        def _series(self):
            return self._prices


    @pd.api.extensions.register_series_accessor(name)
    class SeriesAccessor(SeriesMethods):
        def __init__(self, series):
            self._series = series


register_pandas()


In [3]:
from mintalib.indicators import SMA

prices  = sample_prices()

result = prices.assign(
    sma50 = SMA(50),
)
result

,open,high,low,close,volume,sma50
date,,,,,,
1980-12-12,0.098943,0.099373,0.098943,0.098943,469033600,NaN
1980-12-15,0.094211,0.094211,0.093781,0.093781,175884800,NaN
1980-12-16,0.087328,0.087328,0.086898,0.086898,105728000,NaN
1980-12-17,0.089049,0.089479,0.089049,0.089049,86441600,NaN
1980-12-18,0.091630,0.092061,0.091630,0.091630,73449600,NaN
...,...,...,...,...,...,...
2024-10-15,233.610001,237.490005,232.369995,233.850006,64751400,224.138625
2024-10-16,231.600006,232.119995,229.839996,231.779999,34082200,224.634417
2024-10-17,233.429993,233.850006,230.520004,232.149994,32993800,225.085868


In [4]:
prices.ta.atr()

date
1980-12-12         NaN
1980-12-15         NaN
1980-12-16         NaN
1980-12-17         NaN
1980-12-18         NaN
                ...   
2024-10-15    4.516121
2024-10-16    4.479971
2024-10-17    4.397830
2024-10-18    4.371556
2024-10-21    4.230731
Length: 11056, dtype: float64

In [5]:
prices.close.ta.sma()

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2024-10-15    227.524000
2024-10-16    228.078500
2024-10-17    228.242500
2024-10-18    228.582500
2024-10-21    229.082999
Length: 11056, dtype: float64

In [6]:
prices.close.ta.sma(50)

date
1980-12-12           NaN
1980-12-15           NaN
1980-12-16           NaN
1980-12-17           NaN
1980-12-18           NaN
                 ...    
2024-10-15    224.138625
2024-10-16    224.634417
2024-10-17    225.085868
2024-10-18    225.524600
2024-10-21    225.934400
Length: 11056, dtype: float64

In [7]:
prices.ta.ema()

AttributeError: 'PricesAccessor' object has no attribute 'ema'

In [8]:
import inspect
def binding_wrapper(func):
    signature = inspect.signature(func)
    def wrapper(*args, **kwargs):
        bound = signature.bind(*args, **kwargs)
        bound.apply_defaults()
        return bound.arguments
    return wrapper

@binding_wrapper
def myfunc(a=1, b=2):
    return dict(a=a, b=b)

myfunc()

{'a': 1, 'b': 2}